In [1]:
import warnings
warnings.filterwarnings('ignore')

---

In [2]:
%%capture
!pip install datasets

In [3]:
from datasets import load_dataset

df = load_dataset("shangrilar/ko_text2sql", "origin")['test']
df = df.to_pandas()

df.shape

README.md:   0%|          | 0.00/281 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/25.6M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/61.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/38246 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/112 [00:00<?, ? examples/s]

(112, 4)

In [4]:
!ls -al /root/.cache/huggingface/hub/

total 16
drwxr-xr-x 4 root root 4096 Jan 10 18:29 .
drwxr-xr-x 4 root root 4096 Jan 10 18:29 ..
drwxr-xr-x 6 root root 4096 Jan 10 18:29 datasets--shangrilar--ko_text2sql
drwxr-xr-x 3 root root 4096 Jan 10 18:29 .locks


In [5]:
import os

hf_cache_dir = os.path.expanduser("~/.cache/huggingface/hub/")

model_path = "datasets--shangrilar--ko_text2sql"
model_cache_dir = os.path.join(hf_cache_dir, model_path)

file_paths = []
for root, dirs, files in os.walk(model_cache_dir):
    for file in files:
        if file == "README.md":
            file_paths.append(os.path.join(root, file))

print("README file paths:", file_paths)

README file paths: ['/root/.cache/huggingface/hub/datasets--shangrilar--ko_text2sql/snapshots/ac535d1bb79d4e642f5760ac9596773dc154cb77/README.md']


In [6]:
! cat {file_paths[0]}

---
configs:
- config_name: origin
  data_files:
  - split: train
    path: "origin/train.csv"
  - split: test
    path: "test.csv"
- config_name: clean
  data_files:
  - split: train
    path: "clean/train.csv"
  - split: test
    path: "test.csv"
---

---
license: cc-by-4.0
---


In [7]:
df.sample(5)

,db_id,context,question,answer
96,1,CREATE TABLE clans (\n clan_id INT PRIMARY KE...,"각 클랜의 이름이 몇 글자인지 조회하고, 그 글자 수에 따라 분류하여 각 글자 수별...","SELECT LENGTH(name) as name_length, COUNT(*) F..."
10,1,CREATE TABLE characters (\n character_id INT ...,"장비를 장착한 모든 캐릭터의 아이디, 이름, 아이템명, 레벨을 알려주세요.","SELECT T2.character_id, T2.name, T1.item_name,..."
84,1,CREATE TABLE inventory_items (\n inventory_id...,"인벤토리 아이디가 100, 200, 300에 해당하는 아이템의 이름을 알고 싶어.",SELECT item_name FROM inventory_items WHERE in...
33,1,CREATE TABLE characters (\n character_id INT ...,experience가 1000000 이상이거나 character_class가 'As...,"SELECT player_id, name FROM characters WHERE e..."
46,1,CREATE TABLE players (\n player_id INT PRIMAR...,각 플레이어가 보유한 캐릭터들의 총 레벨을 찾으세요.,"SELECT SUM(T2.level), T1.player_id FROM player..."


---

In [8]:
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt

In [9]:
temp = make_prompt(df.iloc[0]['context'], df.iloc[0]['question'], df.iloc[0]['answer'])
print(temp)

당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
CREATE TABLE quests (
  quest_id INT PRIMARY KEY AUTO_INCREMENT,
  name VARCHAR(255) NOT NULL,
  description TEXT,
  reward_experience INT NOT NULL,
  reward_items VARCHAR(255)
);

### Question:
각 보상 아이템별로 보상 경험치의 합을 구해줘

### SQL:
SELECT reward_items, SUM(reward_experience) AS 보상_경험치_합 FROM quests GROUP BY reward_items;


In [10]:
import json
import pandas as pd
from pathlib import Path

def make_requests_for_gpt_evaluation(df, filename, dir='requests'):
  prompts = []
  for idx, row in df.iterrows():
      prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Question.
If gen_sql and gt_sql do equal job, return "yes" else return "no".
Output JSON Format: {"resolve_yn": ""}""" + f"""

DDL: {row['context']}
Question: {row['question']}
gt_sql: {row['answer']}
gen_sql: {row['gen_sql']}"""
)

  jobs = [{
      "model": "gpt-4-turbo-preview",
      "response_format": { "type": "json_object" },
      "messages": [{"role": "system", "content": prompt}]} for prompt in prompts]

  if not Path(dir).exists():
      Path(dir).mkdir(parents=True)

  with open(Path(dir, filename), "w") as f:
      for job in jobs:
          json_string = json.dumps(job)
          f.write(json_string + "\n")

In [11]:
import os
os.environ["OPENAI_API_KEY"] = "자신의 OpenAI API 키 입력"

# python api_request_parallel_processor.py \
#   --requests_filepath {요청 파일 경로} \
#   --save_filepath {생성할 결과 파일 경로} \
#   --request_url https://api.openai.com/v1/chat/completions \
#   --max_requests_per_minute 300 \
#   --max_tokens_per_minute 100000 \
#   --token_encoding_name cl100k_base \
#   --max_attempts 5 \
#   --logging_level 20


# python examples/api_request_parallel_processor.py \
#   --requests_filepath examples/data/example_requests_to_parallel_process.jsonl \
#   --save_filepath examples/data/example_requests_to_parallel_process_results.jsonl \
#   --request_url https://api.openai.com/v1/embeddings \
#   --max_requests_per_minute 1500 \
#   --max_tokens_per_minute 6250000 \
#   --token_encoding_name cl100k_base \
#   --max_attempts 5 \
#   --logging_level 20

In [12]:
def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
    prompts = []
    responses = []
    with open(input_file, 'r') as json_file:
        for data in json_file:
            prompts.append(json.loads(data)[0]['messages'][0]['content'])
            responses.append(json.loads(data)[1]['choices'][0]['message']['content'])

    df = pd.DataFrame({prompt_column: prompts, response_column: responses})
    df.to_csv(output_file, index=False)
    return df

---

In [13]:
# GPU 설정 후 아래와 같이 최신 버전으로 설치 필요
%%capture
! pip install transformers bitsandbytes accelerate peft

In [14]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

def make_inference_pipeline(model_id):
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
  return pipe

model_id = 'beomi/Yi-Ko-6B'
hf_pipe = make_inference_pipeline(model_id)

example = """당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
CREATE TABLE players (
  player_id INT PRIMARY KEY AUTO_INCREMENT,
  username VARCHAR(255) UNIQUE NOT NULL,
  email VARCHAR(255) UNIQUE NOT NULL,
  password_hash VARCHAR(255) NOT NULL,
  date_joined DATETIME NOT NULL,
  last_login DATETIME
);

### Question:
사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.

### SQL:
"""

hf_pipe(example, do_sample=False, return_full_text=False, max_length=512, truncation=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/9.51k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/643M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Device set to use cuda:0


[{'generated_text': "SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE 

In [15]:
!mkdir results

In [16]:
from datasets import load_dataset

df = load_dataset("shangrilar/ko_text2sql", "origin")['test']
df = df.to_pandas()

for idx, row in df.iterrows():
  prompt = make_prompt(row['context'], row['question'])
  df.loc[idx, 'prompt'] = prompt

gen_sqls = hf_pipe(df['prompt'].tolist(), do_sample=False, return_full_text=False, max_length=512, truncation=True)
gen_sqls = [x[0]['generated_text'] for x in gen_sqls]
df['gen_sql'] = gen_sqls

eval_filepath = "text2sql_evaluation.jsonl"
make_requests_for_gpt_evaluation(df, eval_filepath)

In [17]:
%%capture
! pip install tiktoken

In [18]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."

In [21]:
!python api_request_parallel_processor.py \
--requests_filepath requests/{eval_filepath}  \
--save_filepath results/{eval_filepath} \
--request_url https://api.openai.com/v1/chat/completions \
--max_requests_per_minute 2500 \
--max_tokens_per_minute 100000 \
--token_encoding_name cl100k_base \
--max_attempts 5 \
--logging_level 20

INFO:root:Starting request #0
INFO:root:Starting request #1
INFO:root:Starting request #2
INFO:root:Starting request #3
INFO:root:Starting request #4
INFO:root:Starting request #5
INFO:root:Starting request #6
INFO:root:Starting request #7
INFO:root:Starting request #8
INFO:root:Starting request #9
INFO:root:Starting request #10
INFO:root:Starting request #11
INFO:root:Starting request #12
INFO:root:Starting request #13
INFO:root:Starting request #14
INFO:root:Starting request #15
INFO:root:Starting request #16
INFO:root:Starting request #17
INFO:root:Starting request #18
INFO:root:Starting request #19
INFO:root:Starting request #20
INFO:root:Starting request #21
INFO:root:Starting request #22
INFO:root:Starting request #23
INFO:root:Starting request #24
INFO:root:Starting request #25
INFO:root:Starting request #26
INFO:root:Starting request #27
INFO:root:Starting request #28
INFO:root:Starting request #29
INFO:root:Starting request #30
INFO:root:Starting request #31
INFO:root:Starting

In [22]:
base_eval = change_jsonl_to_csv(f"results/{eval_filepath}", "results/yi_ko_6b_eval.csv", "prompt", "resolve_yn")
base_eval['resolve_yn'] = base_eval['resolve_yn'].apply(lambda x: json.loads(x)['resolve_yn'])
num_correct_answers = base_eval.query("resolve_yn == 'yes'").shape[0]
num_correct_answers

21